In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.metrics import mean_squared_error,r2_score,roc_curve,auc,precision_recall_curve
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold
from sklearn.ensemble import VotingRegressor, VotingClassifier, StackingRegressor, StackingClassifier, GradientBoostingRegressor,GradientBoostingClassifier, BaggingRegressor,BaggingClassifier,RandomForestRegressor,RandomForestClassifier,AdaBoostRegressor,AdaBoostClassifier
from sklearn.linear_model import LinearRegression,LogisticRegression, LassoCV, RidgeCV, ElasticNetCV
from sklearn.neighbors import KNeighborsRegressor
import itertools as it
import time as time
import xgboost as xgb

In [2]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test (1).csv
Saving train.csv to train (1).csv


In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
nan_cols = test.isna().sum()
test1 = test.drop(nan_cols[nan_cols > 269].index.tolist(), axis = 1)
train1 = train.drop(nan_cols[nan_cols > 269].index.tolist(), axis = 1)

In [4]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=3)

# fit and transform the data
test_imputed = pd.DataFrame(imputer.fit_transform(test1), columns=test1.columns)
train_imputed = pd.DataFrame(imputer.fit_transform(train1), columns=train1.columns)

In [5]:
X = train_imputed.drop('y', axis = 1)
y = train_imputed.y

X_test = test_imputed

In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)
X_test_scaled = scaler.transform(X_test)

### Tuning

In [7]:
#K-fold cross validation to find optimal parameters for XGBoost
start_time = time.time()
param_grid = {'max_depth': [4,6,8],
              'learning_rate': [0.01,0.05,0.1],
               'reg_lambda':[0,0.01,0.001],
                'n_estimators':[500,1000,2500]}

cv = KFold(n_splits=5,shuffle=True,random_state=1)
optimal_params = GridSearchCV(estimator=xgb.XGBRegressor(random_state=1),                                                       
                             param_grid = param_grid,                             
                             verbose = 1,
                             n_jobs=-1,
                             cv = cv)
optimal_params.fit(X_scaled,y)  
print("Optimal parameter values =", optimal_params.best_params_)
print("Optimal cross validation R-squared = ",optimal_params.best_score_)
print("Time taken = ", (time.time()-start_time)/60, " minutes")

Fitting 5 folds for each of 81 candidates, totalling 405 fits
Optimal parameter values = {'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 500, 'reg_lambda': 0.001}
Optimal cross validation R-squared =  0.38527059015418336
Time taken =  204.0742035150528  minutes


In [12]:
model = xgb.XGBRegressor(random_state=1,max_depth=4,n_estimators=500,
                                         learning_rate = 0.05,reg_lambda=0.05)
model.fit(X_scaled,y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=4, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=500, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=1, ...)

In [23]:
importances = pd.DataFrame()
importances['Predictor'] = list(X.columns)
importances['Importance'] = list(model.feature_importances_)
importances = importances.sort_values(by='Importance', ascending=False)
importances[0:10]

,Predictor,Importance
563,x585,0.033838
142,x146,0.028513
240,x248,0.018459
110,x114,0.017480
486,x506,0.016860
435,x454,0.016428
402,x420,0.016191
112,x116,0.014846
615,x640,0.013948
236,x243,0.011677


In [24]:
important = ['x585','x146','x248','x114','x506','x454','x420','x116','x640','x243']

In [ ]:
pred = pd.DataFrame()
pred['id'] = test['id']
pred['y'] = model.predict(X_test)
pred = pred.set_index(['id'])
pred.to_csv('Witarsa_Ashley_regression.csv') 